In [1]:
import pandas as pd 
import numpy as np 

In [3]:
train = pd.read_csv("train.csv")
print("Train Shape : " ,train.shape)
train.head()

Train Shape :  (517754, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [5]:
test = pd.read_csv("test.csv")
test["accident_risk"] = 0.5
print("Test Shape :" , test.shape)
test.head() 

Test Shape : (172585, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1,0.5
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0,0.5
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1,0.5
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2,0.5
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3,0.5


In [7]:
orig = []
for k in [2,10,100]:
    df = pd.read_csv(f"synthetic_road_accidents_{k}k.csv")
    orig.append(df)
orig = pd.concat(orig,axis=0)
orig["id"] = np.arange(len(orig))+test["id"].max()+1
orig = orig[train.columns]
print("Original data shape :" , orig.shape)
orig.head()

Original data shape : (112000, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,690339,rural,2,0.72,60,daylight,clear,True,False,aftenoon,False,False,2,0.37
1,690340,highway,4,0.95,45,daylight,foggy,False,True,evening,False,True,1,0.40
2,690341,rural,1,0.72,25,night,rainy,False,False,evening,True,False,1,0.55
3,690342,rural,4,0.86,70,dim,foggy,True,False,morning,True,True,1,0.56
4,690343,highway,1,0.00,60,night,rainy,True,True,morning,True,True,3,0.54


In [9]:
combine = pd.concat([train,test,orig] , axis = 0 , ignore_index = True )
print("Combined Shape : "  , combine.shape)

Combined Shape :  (802339, 14)


In [11]:
combine.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


## Feature Engineering

In [14]:
Features = list(orig.columns[1:-1])
Target = orig.columns[-1]
print(f"Features: {Features} , Target : '{Target}'")

Features: ['road_type', 'num_lanes', 'curvature', 'speed_limit', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season', 'num_reported_accidents'] , Target : 'accident_risk'


## Adding a new Feature

In [17]:
import scipy

###  The Data Generation Logic

The competition data assumes that the "True Risk" (Y) is generated by a linear formula (calculated in f(X)) plus some random noise (ϵ):

Y=μ+ϵ,where ϵ∼N(0,σ 
2
 )
However, probabilities or risk scores must stay between 0 and 1. Therefore, the final target (T) is "clipped":

If Y<0, then T=0

If Y>1, then T=1

If 0≤Y≤1, then T=Y

### Applying the Optimal Bayesian Formula

In [21]:
def f(X):
    return \
    0.3 * X["curvature"] + \
    0.2 * (X["lighting"] == "night").astype(int) + \
    0.1 * (X["weather"] != "clear").astype(int) + \
    0.2 * (X["speed_limit"]>=60).astype(int) + \
    0.1 * (X["num_reported_accidents"] > 2).astype(int)

def clip(f):
    def clip_f(X):
        sigma = 0.05
        mu = f(X)
        a,b = -mu/sigma , (1-mu)/sigma
        Phi_a , Phi_b = scipy.stats.norm.cdf(a) , scipy.stats.norm.cdf(b)
        phi_a , phi_b = scipy.stats.norm.pdf(a) , scipy.stats.norm.pdf(b)
        return mu*(Phi_a - Phi_b) + sigma*(phi_a - phi_b) +1 - Phi_b
    return clip_f

z = clip(f)(combine)
combine["y"] = z.values
Features.append("y")

### The code returns:

Result=μ⋅(Φ(b)−Φ(a))+σ⋅(ϕ(a)−ϕ(b))+1⋅(1−Φ(b))
This formula is the sum of three parts:

The Middle Part: The expected value of the distribution specifically between 0 and 1.

The Lower Tail: Values below 0 are clipped to 0, so they contribute 0 to the expectation (this is why you don't see a term for Φ(a) multiplied by anything).

The Upper Tail: Values above 1 are clipped to 1. The probability of being above 1 is (1−Φ(b)), so we add 1×(1−Φ(b)).

###  Why do we need the clip function?

If you just use the linear result μ from f(X), your prediction will be biased. For example, if μ=0.01 and there is noise, many samples will fall below 0 and get clipped to 0. The average value you actually observe in the data will be slightly higher than 0.01 because the "negative" errors are all stuck at 0, while "positive" errors can move up.

The clip function calculates the Expected Value (E[T]) of that clipped normal distribution.

## Identify Nums and Cats

In [26]:
Cats = []
Nums = []
for c in Features :
    t = "CAT"
    if combine[c].dtype =='object':
        Cats.append(c)
    else :
        Nums.append(c)
        t = "NUM"
    n = combine[c].nunique()
    na = combine[c].isna().sum()
    print(f"[{t}] {c} has {n} unique and {na} NA")
print("Cats : " , Cats)
print("Nums : ", Nums)

[CAT] road_type has 3 unique and 0 NA
[NUM] num_lanes has 4 unique and 0 NA
[NUM] curvature has 298 unique and 0 NA
[NUM] speed_limit has 5 unique and 0 NA
[CAT] lighting has 3 unique and 0 NA
[CAT] weather has 3 unique and 0 NA
[NUM] road_signs_present has 2 unique and 0 NA
[NUM] public_road has 2 unique and 0 NA
[CAT] time_of_day has 4 unique and 0 NA
[NUM] holiday has 2 unique and 0 NA
[NUM] school_season has 2 unique and 0 NA
[NUM] num_reported_accidents has 11 unique and 0 NA
[NUM] y has 1094 unique and 0 NA
Cats :  ['road_type', 'lighting', 'weather', 'time_of_day']
Nums :  ['num_lanes', 'curvature', 'speed_limit', 'road_signs_present', 'public_road', 'holiday', 'school_season', 'num_reported_accidents', 'y']


## Label Encode Cats 

Machine learning models like XGBoost cannot directly understand text (like "Rainy" or "Sunny")

In [30]:
Sizes = {}
for c in Cats:
    combine[c],_ = combine[c].factorize()
    Sizes[c] = combine[c].max()+1
    combine[c] = combine[c].astype('int32')
    combine[c] = combine[c].astype('int32')

print("Cardianility of all Cats :" , Sizes)

Cardianility of all Cats : {'road_type': 3, 'lighting': 3, 'weather': 3, 'time_of_day': 4}


In [32]:
train = combine.iloc[:len(train)]
test = combine.iloc[len(train):len(train) + len(test)]
orig = combine.iloc[-len(orig):]


In [34]:
print(f"Train Shape : {train.shape} , Test Shape : {test.shape} and Original Data : {orig.shape}")

Train Shape : (517754, 15) , Test Shape : (172585, 15) and Original Data : (112000, 15)


## Target Encode 

Target Encoding (TE) is a technique where you replace a categorical value (like "Urban" or "Rural") with a number that represents the average (mean) of the target variable for that specific category.

In [38]:
Te  = []
for c in Features :
    tmp = orig.groupby(c)[Target].mean()
    n = f"Te_{c}"
    print(f"{n}, ",end = "")
    tmp.name = n
    train = train.merge(tmp , on = c , how = "left")
    test = test.merge(tmp , on = c , how = "left")
    Te.append(n)


Te_road_type, Te_num_lanes, Te_curvature, Te_speed_limit, Te_lighting, Te_weather, Te_road_signs_present, Te_public_road, Te_time_of_day, Te_holiday, Te_school_season, Te_num_reported_accidents, Te_y, 

#### Lill Theory 

How the "Average" is calculated

The code line tmp = orig.groupby(c)[TARGET].mean() does the math:

Groupby: It puts all rows with the same category together. For example, all rows where weather is "Rainy".

Target: It looks at the accident_risk (the target) for all those "Rainy" rows.

Mean: It adds up all those risk scores and divides by the number of rows.

Concrete Example: Imagine the Original dataset has 3 rows for "Rainy" weather:

Row A: Risk = 0.2

Row B: Risk = 0.4

Row C: Risk = 0.3

Calculation: (0.2+0.4+0.3)/3=0.3

Result: Every time the word "Rainy" appears in your train or test set, the new column TE_weather will contain the value 0.3.

### Why use the orig data for the average?

Usually, if you calculate the average using the train set itself, you risk overfitting (the model might just memorize the specific labels).

By using the Original (external) dataset to calculate these averages:

You get a "clean" statistical average from a different population of data.

It acts as a global prior. It tells the model what the "normal" risk for a road type is, which helps the model stay stable even if the competition's training set has some weird outliers.

## Train XGBoost on Residuals 

In [44]:
from sklearn.model_selection import KFold
import xgboost as xgb
print("XGB Version : ", xgb.__version__)

XGB Version :  3.0.5


In [46]:
Folds = 7 
Seed = 42
params = {
    "objective" : "reg:squarederror" ,
    "eval_metric" : "rmse" ,
    "learning_rate" : 0.01 ,
    "max_depth" : 6 ,
    "subsample" : 0.9 , 
    "colsample_bytree" : 0.6 ,
    "seed" : Seed
}

Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
range: (0,1] <br>
Colsample_bytree is the subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.

In [53]:
oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))
kf = KFold(n_splits = Folds , 
          shuffle = True , 
          random_state = Seed)
for fold , (train_idx , val_idx) in enumerate(kf.split(train)):
    print("#"*25)
    print(f"### Fold {fold+1} ###")
    print("#"*25)

    X_train = train.iloc[train_idx][Features+Te].copy()
    ## Residual Training = target− baseline guess(y) instead of just y
    y_train = train.iloc[train_idx][Target] - train.iloc[train_idx]['y']

    X_valid = train.iloc[val_idx][Features+Te].copy()
    y_valid = train.iloc[val_idx][Target] - train.iloc[val_idx]['y']
    y_valid2= train.iloc[val_idx]['y'].values

    X_test = test[Features + Te].copy()
    y_test2 = test['y'].values 

    dtrain = xgb.DMatrix(X_train ,
                         label = y_train ,
                         enable_categorical = True)
    
    dval = xgb.DMatrix(X_valid ,
                       label = y_valid ,
                       enable_categorical = True)
    
    dtest = xgb.DMatrix(X_test ,
                        enable_categorical = True)

    model = xgb.train(
        params = params ,
        dtrain = dtrain , 
        num_boost_round = 100_100 , 
        evals = [(dtrain , "train") ,(dval , "valid")],
        early_stopping_rounds = 200,
        verbose_eval = 200
    )

    oof_preds[val_idx] = model.predict(dval,iteration_range = (0, model.best_iteration + 1)) + y_valid2
    test_preds += (model.predict(dtest , iteration_range = (0 ,model.best_iteration+1)) + y_test2) / Folds

#########################
### Fold 1 ###
#########################
[0]	train-rmse:0.32165	valid-rmse:0.32135
[200]	train-rmse:0.07188	valid-rmse:0.07174
[400]	train-rmse:0.05662	valid-rmse:0.05663
[600]	train-rmse:0.05607	valid-rmse:0.05615
[800]	train-rmse:0.05591	valid-rmse:0.05604
[1000]	train-rmse:0.05581	valid-rmse:0.05600
[1200]	train-rmse:0.05573	valid-rmse:0.05596
[1400]	train-rmse:0.05566	valid-rmse:0.05594
[1600]	train-rmse:0.05559	valid-rmse:0.05593
[1800]	train-rmse:0.05553	valid-rmse:0.05591
[2000]	train-rmse:0.05548	valid-rmse:0.05590
[2200]	train-rmse:0.05543	valid-rmse:0.05590
[2400]	train-rmse:0.05538	valid-rmse:0.05589
[2600]	train-rmse:0.05533	valid-rmse:0.05589
[2800]	train-rmse:0.05528	valid-rmse:0.05589
[3000]	train-rmse:0.05524	valid-rmse:0.05589
[3036]	train-rmse:0.05523	valid-rmse:0.05589
#########################
### Fold 2 ###
#########################
[0]	train-rmse:0.32184	valid-rmse:0.32022
[200]	train-rmse:0.07183	valid-rmse:0.07227
[400]	train-rmse:0.056

num_boost_round=100_000: This allows the model to build up to 100,000 trees<br>
early_stopping_rounds=200: If the validation score (RMSE) doesn't improve for 200 consecutive trees, the model stops training to prevent overfitting

## Cv Score 

In [56]:
m = np.sqrt(np.mean((oof_preds - train[Target].values)**2))
print(f"Overall CV Score :{m}")
np.save(f"oof" , oof_preds)

Overall CV Score :0.0559668356800143


In [58]:
m = np.sqrt(np.mean((train.y.values - train[Target].values)**2))
print("Baseline CV RMSE = " , m )


Baseline CV RMSE =  0.7750977593312632


## Submission File Generation

In [69]:
sub = pd.read_csv("sample_submission.csv")
sub["accident_risk"] = test_preds
sub.to_csv("submission.csv" ,  index = False)
sub.head()

,id,accident_risk
0,517754,0.294037
1,517755,0.119924
2,517756,0.180130
3,517757,0.313486
4,517758,0.395013
